In [ ]:
# import the wrapper
from p21cmfastwrapper import *
from power_spectra_1D import *
from power_spectra_2D import *

In [ ]:
sim = Simulation(save_ondisk=False, save_inclass=True, write_cache=True)
params = {"max_redshift": 15, "random_seed": 1, "astro_params": {"HII_EFF_FACTOR": 29}} 
#sim.run_lightcone(kargs=params)

In [ ]:
params = {"astro_params": {"HII_EFF_FACTOR": [2,8], "NU_X_THRESH": [300,800], "ION_Tvir_MIN": [4,5]}}
sim.generate_range(params, (lambda a,b: np.random.uniform(a,b)))

In [ ]:
data = sim.data[0]

k_bins = np.array([0.18137994, 0.2720699 , 0.36275987,
0.45344984, 0.54413981, 0.63482978, 0.72551975, 0.81620971,
0.90689968, 0.99758965, 1.08827962, 1.17896959, 1.26965955,
1.36034952, 1.45103949, 1.54172946, 1.63241943, 1.7231094 ,
1.81379936, 1.90448933, 1.9951793 , 2.08586927, 2.17655924,
2.26724921, 2.35793917, 2.44862914, 2.53931911, 2.63000908,
2.72069905, 2.81138901, 2.90207898, 2.99276895, 3.08345892,
3.17414889, 3.26483886, 3.35552882, 3.44621879])

zbins = np.linspace(0,len(sim.data[0].lightcone_redshifts)-1,8).astype(int)

# create an tensor to store the data (z_bin, (ps, k, var_ps))
k_shape = len(k_bins)
ps = np.empty((len(zbins)-1, 3, k_shape-1))

for bin in range(len(zbins) - 1):
    print(bin)
    physical_size = data.lightcone_distances[zbins[bin+1]] - data.lightcone_distances[zbins[bin]]
    ps[bin,:,:] = get_power(deltax= data.brightness_temp[:,:,zbins[bin]:zbins[bin+1]], 
                    boxlength=(*data.lightcone_dimensions[:2], physical_size), bin_ave=True, 
                    ignore_zero_mode=True, get_variance=True, bins=k_bins, vol_normalised_power=True)
    ps[bin,0,:] *= ps[bin,1,:]**3/(2* np.pi**2)

In [ ]:
test = {"main": {"sek":0, "sek2":0}}
print(test)

In [ ]:
fill_dict(test, [1,2])

In [ ]:
def fill_dict(nested_dict, array, index=0):
    """
    Fills the nested dictionary with elements from the array recursively.

    Parameters:
    nested_dict (dict): The nested dictionary to be filled.
    array (list): The list of elements to fill into the nested dictionary.
    index (int): The current index in the array.

    Returns:
    int: The next index to be used in the array.
    """
    for key in nested_dict:
        if isinstance(nested_dict[key], dict):
            index = fill_dict(nested_dict[key], array, index)
        else:
            if index < len(array):
                nested_dict[key] = array[index]
                index += 1
            else:
                break
    return nested_dict

In [ ]:
import matplotlib.image as mpimg
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as pl

image = sim.data[0].brightness_temp[:,:,-1]

npix = image.shape[0]

fourier_image = np.fft.fftn(image)
fourier_amplitudes = np.abs(fourier_image)**2

kfreq = np.fft.fftfreq(npix) * npix
kfreq2D = np.meshgrid(kfreq, kfreq)
knrm = np.sqrt(kfreq2D[0]**2 + kfreq2D[1]**2)

knrm = knrm.flatten()
fourier_amplitudes = fourier_amplitudes.flatten()

kbins = np.arange(0.5, npix//2+1, 1.)
kvals = 0.5 * (kbins[1:] + kbins[:-1])
Abins, _, _ = stats.binned_statistic(knrm, fourier_amplitudes,
                                     statistic = "mean",
                                     bins = kbins)
Abins *= np.pi * (kbins[1:]**2 - kbins[:-1]**2)

pl.loglog(kvals, Abins)
pl.xlabel("$k$")
pl.ylabel("$P(k)$")
pl.tight_layout()
pl.savefig("cloud_power_spectrum.png", dpi = 300, bbox_inches = "tight")

In [3]:
from mcmc import *

init_params_ranges = {"astro_params": {"ION_Tvir_MIN": [4,5.3], "HII_EFF_FACTOR": [5, 100], 
                    "L_X": [30,50], "NU_X_THRESH": [100, 1500]}}

def log_prior(theta):
        T_vir, H_eff, LX, nu_x = theta
        if 5 < H_eff < 100 and 100 < nu_x < 1500 and 4 < T_vir < 5.3 and 30 < LX < 50:
            return 0
        return - np.inf
    
def log_likelihood(t,f):
    return - np.sum((t - f)**2/f)

mcrun = mcmc(nwalker=8, z_bins = 11, debug = True, log_likelihood=log_likelihood, prior=log_prior)
mcrun.make_fiducial(load = False)

mcrun.run_aies(init_params_ranges)

use astro_params default config: True
use cosmo_params default config: True
use user_params default config: False
use flag_options default config: False
use global_params default config: True
Using 21cmFAST version 3.3.1


/home/tom/miniconda3/lib/python3.12/site-packages/py21cmfast/_utils.py:400: UserWarning: The following parameters to FlagOptions are not supported: ['USE_VELS_AUX']
  warnings.warn(


[[   4.38828343   20.72040763   44.34810243  314.06696938]
 [   4.39486127   84.53812601   34.41690317 1273.25146646]
 [   4.09079477   29.766908     38.64665543  474.70146236]
 [   4.94411614   24.58446714   49.25839262  377.84908259]
 [   4.98367478   67.15492021   40.65710558  224.04599141]
 [   5.18998718   18.42108385   33.80680821  597.44454547]
 [   4.59609238   45.26134953   32.63791794  713.86143423]
 [   4.99362293   64.23088496   43.95400808  886.82117963]]


In [1]:
from mcmc import *
init_params_ranges = {"astro_params": {"ION_Tvir_MIN": [4,5.3], "HII_EFF_FACTOR": [5, 100], 
                    "L_X": [30,50], "NU_X_THRESH": [100, 1500]}}

# input: init cube for parameter
def prior_sampling(theta):
        T_vir, H_eff, LX, nu_x = theta
        T_vir = 4 + 1.3*T_vir
        H_eff = 5 + 95*H_eff
        LX = 30 + 20*LX
        nu_x = 100 + 1400*nu_x
        return np.array([T_vir, H_eff, LX, nu_x])
    
# returns samples from the posterior
    
def log_likelihood(t,f):
    return - np.sum((t - f)**2/f)


mcrun = mcmc(z_bins = 11, debug = True, log_likelihood=log_likelihood, prior=prior_sampling)
mcrun.make_fiducial(load = True)

mcrun.run_ns(init_params_ranges)


/home/tom/miniconda3/lib/python3.12/site-packages/py21cmfast/_cfg.py:57: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn(
/home/tom/miniconda3/lib/python3.12/site-packages/py21cmfast/_cfg.py:41: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn("Your configuration file is out of date. Updating...")
[potato:37808] shmem: mmap: an error occurred while determining whether or not /tmp/ompi.potato.1000/jf.0/1918173184/shared_mem_cuda_pool.potato could be created.
[potato:37808] create_and_attach: unable to create shared memory BTL coordinating structure :: size 134217728 
/home/tom/miniconda3/lib/python3.12/site-packages/py21cmfast/_utils.py:400: UserWarning: The following parameters to FlagOptions are not supported: ['USE_VELS_AUX']
  warnings.warn(


use astro_params default config: True
use cosmo_params default config: True
use user_params default config: False
use flag_options default config: False
use global_params default config: True
Using 21cmFAST version 3.3.1
 MultiNest Warning: no resume file found, starting from scratch
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  400
 dimensionality =    4
 *****************************************************
 Starting MultiNest
 generating live points
lprob=-414.54623843494124
lprob=-129.71738307052564


Exception ignored on calling ctypes callback function: <function run.<locals>.loglike at 0x73a7382aeca0>
Traceback (most recent call last):
  File "/home/tom/miniconda3/lib/python3.12/site-packages/pymultinest/run.py", line 223, in loglike
    return LogLikelihood(cube, ndim, nparams, lnew)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/tom/miniconda3/lib/python3.12/site-packages/pymultinest/solve.py", line 56, in SafeLoglikelihood
    l = float(LogLikelihood(a))
              ^^^^^^^^^^^^^^^^
  File "/home/tom/Documents/projects/master/21cm-wrapper/mcmc.py", line 56, in p_wrapper
    test_cone = self.run_lightcone(kargs=run_params, commit=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/tom/Documents/projects/master/21cm-wrapper/p21cmfastwrapper.py", line 144, in run_lightcone
    run = p21c.run_lightcone(**self.input_params)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/tom/miniconda3/lib/python3.12/site-packag

In [2]:
from mcmc import *
mcrun = mcmc(nwalker=8, z_bins = 11, debug = True, log_likelihood=log_likelihood, prior=log_prior)
mcrun.make_fiducial(load = False)

/home/tom/miniconda3/lib/python3.12/site-packages/py21cmfast/_cfg.py:57: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn(
/home/tom/miniconda3/lib/python3.12/site-packages/py21cmfast/_cfg.py:41: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn("Your configuration file is out of date. Updating...")
[potato:37957] shmem: mmap: an error occurred while determining whether or not /tmp/ompi.potato.1000/jf.0/997588992/shared_mem_cuda_pool.potato could be created.
[potato:37957] create_and_attach: unable to create shared memory BTL coordinating structure :: size 134217728 


NameError: name 'log_likelihood' is not defined

In [ ]:
import py21cmfast
py21cmfast.inputs.CosmoParams._defaults_

In [ ]:
args = {"astro_params": 
        {"NU_X_THRESH": [400,700], 
        "HII_EFF_FACTOR": [10,100],
        "L_X": [1,2],
        "ION_Tvir_MIN": [4,5]},
    "cosmo_params":
        {"OMm": [0,1]},
    "global_params": 
        {"M_WDM": [1,2]}}

In [ ]:
py21cmfast.inputs.global_params.M_WDM

In [ ]:
test = {"main": {"blub": 1, "blubber": 2}, "sek": {"schwubble": {"swap": 4}}}

In [ ]:
def extract_keys(nested_dict):
    keys = []
    for key in nested_dict:
        keys.append(key)
        if isinstance(nested_dict[key], dict):
            keys.extend(extract_keys(nested_dict[key]))
    return keys

In [ ]:
extract_keys(test)